<a href="https://colab.research.google.com/github/RodrigoRoman/ml_ai_portafolio/blob/main/long_short_term_memory/LSTM_with_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>LSTM</h1>

In [ ]:
import numpy as np
import random

In [ ]:
from google.colab import drive

In [ ]:
news_s_path = "/content/drive/MyDrive/newsSpace"

<h2>Data preprocessing functions</h2>

In [ ]:
import re
def is_url(s):
    # A simple regex to check for a basic URL structure
    return re.match(r'https?://', s) is not None
def tokenize_article(line):
  url_index = next((i for i, item in enumerate(line) if is_url(item)), None)
  if url_index is not None:
    return re.split(r'[ ,.;:!?()]+', ' '.join(line[url_index+1:]))
  return None


def process_file(filepath, num_articles):
  articles = []
  vocabulary = set()
  pattern = re.compile(r'[ ,.;:!?()]+')
  word_pattern = re.compile(r"\b[A-Za-z]+'?[A-Za-z]*(?=\s|\b)")
  try:
    with open(filepath, encoding='ISO-8859-1') as file:
      data = file.read()
      pattern = re.compile(r"\((Reuters|AP)\)[\t\n]+(.*?)[\t\n]+\d+[\t\n]+[0-9]{4}-[0-9]{2}-[0-9]{2}", re.DOTALL)
      raw_articles = pattern.findall(data)
      print("amount of articles")
      print(len(raw_articles))

      for article in raw_articles:
        if len(articles) < num_articles:
          article_text = article[1].strip()
          # Cleaning and processing the article text
          words = word_pattern.findall(article_text.lower())
          # cleaned_article = ' '.join(words)
          articles.append(words)
          # Update vocabulary
          vocabulary.update(words)
  except IOError:
    print("Error opening or reading the file")
    return [], set()
  return articles, vocabulary

<h2>DataLoading</h2>

In [ ]:
num_articles = 10
news_s_path = "/content/drive/MyDrive/newsSpace"

data_articles, vocabulary = process_file(news_s_path, num_articles)
print(data_articles)

Error opening or reading the file
[]


<h3>Split data into train and test</h3>
<p>Split data into training and testing sets, and further into input and target pairs where the target is the next word</p>

In [ ]:

# Preprocess articles to create input-target pairs
def create_input_target(articles):
  article_targets = []
  for article in articles:
    target_article = []
    for i in range(len(article) - 1):
      target_word = article[i + 1]
      target_article.append(target_word)
    article_targets.append(target_article)
  return article_targets

# Split data into training and test sets
def split_data(data, test_percentage):
  split_point = int(len(data) * test_percentage)
  test_set = data[:split_point]
  training_set = data[split_point:]
  return training_set, test_set


# Example usage
test_percentage = 0.2

# Create input-target pairs
article_targets  = create_input_target(data_articles)
print("Input target pairs")
print(article_targets)

# Vocabulary
word_to_idx = {ch:i for (i,ch) in enumerate(list(vocabulary))}
idx_to_word = {i:ch for (i,ch) in enumerate(list(vocabulary))}

# Take input-target as x and y
x_train_words, x_test_words = split_data(data_articles,test_percentage)
y_train_words, y_test_words = split_data(article_targets,test_percentage)

# Change the data to their index versions
x_train = [[word_to_idx[word] for word in article if word in word_to_idx] for article in x_train_words]
y_train = [[word_to_idx[word] for word in article if word in word_to_idx] for article in y_train_words]
x_test = [[word_to_idx[word] for word in article if word in word_to_idx] for article in x_test_words]
y_test = [[word_to_idx[word] for word in article if word in word_to_idx] for article in y_test_words]

Input target pairs
[]


<h2>LSTM</h2>

In [ ]:
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
class LSTM:
    def __init__(self, hidden_size, vocab_size, learning_rate):
      self.hidden_size = hidden_size
      self.vocab_size = vocab_size
      self.learning_rate = learning_rate

      # Initialize LSTM parameters
      #Forget gate
      self.Wf = np.random.randn(hidden_size, hidden_size + vocab_size)
      #Input gate
      self.Wi = np.random.randn(hidden_size, hidden_size + vocab_size)
      #Output gate
      self.Wo = np.random.randn(hidden_size, hidden_size + vocab_size)
      self.Wc = np.random.randn(hidden_size, hidden_size + vocab_size)
      self.Wy = np.random.randn(vocab_size, hidden_size)
      self.bf = np.zeros((hidden_size, 1))
      self.bi = np.zeros((hidden_size, 1))
      self.bo = np.zeros((hidden_size, 1))
      self.bc = np.zeros((hidden_size, 1))
      self.by = np.zeros((vocab_size, 1))

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    def cross_entropy(self, probs, targets):
        loss = 0
        epsilon = 1e-9
        for t in range(len(targets)):
          prob = max(probs[t][targets[t]], epsilon)
          loss += -np.log(prob)
        return loss

    def forward(self, inputs, h_prev, c_prev):
        caches = []
        h_next, c_next = h_prev, c_prev
        for t in range(len(inputs)):
            z = np.row_stack((h_next, inputs[t]))
            f = sigmoid(np.dot(self.Wf, z) + self.bf)
            i = sigmoid(np.dot(self.Wi, z) + self.bi)
            o = sigmoid(np.dot(self.Wo, z) + self.bo)
            c_bar = np.tanh(np.dot(self.Wc, z) + self.bc)

            c_next = f * c_next + i * c_bar
            h_next = o * np.tanh(c_next)

            y = np.dot(self.Wy, h_next) + self.by
            p = self.softmax(y)
            caches.append((h_next, c_next, f, i, o, c_bar, z, y, p,c_prev))
        return caches

    def backward(self, caches, targets):
        # Initialize gradients
        dWf, dWi, dWo, dWc, dWy = np.zeros_like(self.Wf), np.zeros_like(self.Wi), np.zeros_like(self.Wo), np.zeros_like(self.Wc), np.zeros_like(self.Wy)
        dbf, dbi, dbo, dbc, dby = np.zeros_like(self.bf), np.zeros_like(self.bi), np.zeros_like(self.bo), np.zeros_like(self.bc), np.zeros_like(self.by)
        dh_next, dc_next = np.zeros_like(caches[0][0]), np.zeros_like(caches[0][1])

        # Backpropagation through time
        for t in reversed(range(len(targets))):
          dh_next, dc_next, dWf_t, dWi_t, dWo_t, dWc_t, dbf_t, dbi_t, dbo_t, dbc_t, dWy_t, dby_t = self.lstm_step_backward(dh_next, dc_next, caches[t], targets[t])
          dWf += dWf_t
          dWi += dWi_t
          dWo += dWo_t
          dWc += dWc_t
          dbf += dbf_t
          dbi += dbi_t
          dbo += dbo_t
          dbc += dbc_t
          dWy += dWy_t
          dby += dby_t
        return dWf, dWi, dWo, dWc, dWy, dbf, dbi, dbo, dbc, dby

    def lstm_step_backward(self, dh_next, dc_next, cache, target):
        # Unpack cache
        h_next, c_next, f, i, o, c_bar, z, y, p,c_prev = cache

        # Gradients of loss with respect to y
        dy = np.copy(p)
        dy[target] -= 1

        # Gradients with respect to Wy and by
        dWy = np.dot(dy, h_next.T)
        dby = dy

        # Gradients with respect to h_next
        dh = np.dot(self.Wy.T, dy) + dh_next

        # Gradients with respect to o
        do = dh * np.tanh(c_next)
        do = do * o * (1 - o)

        # Gradients with respect to c_next
        dc = dh * o * (1 - np.tanh(c_next)**2) + dc_next

        # Gradients with respect to i
        di = dc * c_bar
        di = di * i * (1 - i)

        # Gradients with respect to c_bar
        dc_bar = dc * i
        dc_bar = dc_bar * (1 - c_bar**2)

        # Gradients with respect to f
        df = dc * c_prev
        df = df * f * (1 - f)

        # Gradients with respect to z
        dz = (np.dot(self.Wf.T, df)
              + np.dot(self.Wi.T, di)
              + np.dot(self.Wo.T, do)
              + np.dot(self.Wc.T, dc_bar))

        # Gradients with respect to the weights and biases
        dWf = np.dot(df, z.T)
        dWi = np.dot(di, z.T)
        dWo = np.dot(do, z.T)
        dWc = np.dot(dc_bar, z.T)
        dbf = df
        dbi = di
        dbo = do
        dbc = dc_bar

        # Compute gradients with respect to the previous hidden state and cell state
        dh_prev = dz[:self.hidden_size, :]
        dc_prev = f * dc

        return dh_prev, dc_prev, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWy, dby

    def update_parameters(self, dWf, dWi, dWo, dWc, dWy, dbf, dbi, dbo, dbc, dby):
        # Update the weights and biases using the gradients and the learning rate
        self.Wf -= self.learning_rate * dWf
        self.Wi -= self.learning_rate * dWi
        self.Wo -= self.learning_rate * dWo
        self.Wc -= self.learning_rate * dWc
        self.Wy -= self.learning_rate * dWy
        self.bf -= self.learning_rate * dbf
        self.bi -= self.learning_rate * dbi
        self.bo -= self.learning_rate * dbo
        self.bc -= self.learning_rate * dbc
        self.by -= self.learning_rate * dby

    def train(self, x_train, y_train, epochs):
        print("inside train")
        print(len(x_train))
        for epoch in range(epochs):
            print("Inside epoch")
            h_prev = np.zeros((self.hidden_size, 1))
            c_prev = np.zeros((self.hidden_size, 1))

            for batch_idx, inputs in enumerate(x_train):
                print("inside batch")
                # Convert inputs to one-hot encoded vectors
                inputs_one_hot = [np.eye(self.vocab_size)[:, x].reshape(-1, 1) for x in inputs]

                # Forward pass
                caches = self.forward(inputs_one_hot, h_prev, c_prev)

                # Backward pass
                dWf, dWi, dWo, dWc, dWy, dbf, dbi, dbo, dbc, dby = self.backward(caches, y_train[batch_idx])

                # Update parameters
                self.update_parameters(dWf, dWi, dWo, dWc, dWy, dbf, dbi, dbo, dbc, dby)

                # Update the hidden and cell states
                h_prev, c_prev = caches[-1][0], caches[-1][1]

                # Compute and print the loss for monitoring
                loss = self.cross_entropy([cache[-1] for cache in caches], y_train[batch_idx])
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss}')

<h2>Training process</h2>

In [ ]:
epochs = 800

vocabulary_size = len(vocabulary)
embedding_size = 400
hidden_size = 1000
#  def __init__(self, hidden_size,vocab_size,embedding_size,learning_rate):
#rnn = RNN(hidden_size=hidden_size, vocab_size=vocabulary_size,learning_rate=0.0001)
lstm = LSTM(hidden_size=hidden_size, vocab_size=vocabulary_size,learning_rate=0.01)
print(x_train)
lstm.train(x_train,y_train,epochs)

[]
inside train
0
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside epoch
Inside 